In [1]:
import requests, json
import pandas as pd
from useful_funcs import *
from predict_player_scores import get_latest_players_df

In [2]:
league_url = 'https://draft.premierleague.com/api/league/19188/element-status'
r = requests.get(league_url).json()

In [3]:
ownership_df = pd.json_normalize(r['element_status'])

In [4]:
url = 'https://draft.premierleague.com/api/bootstrap-static'
req = requests.get(url).json()
players_df = pd.json_normalize(req['elements'])

In [5]:
players_df['full_name'] = players_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)
merge_name_df = players_df[['id', 'full_name']]

In [6]:
ownership_df_name = ownership_df.merge(merge_name_df, how='left', left_on='element', right_on='id') 

In [7]:
old_pred_df = pd.read_csv('../drafting/player_score_predictions_delta.csv')

In [8]:
old_pred_df['full_name'] = old_pred_df['full_name'].str.lower().str.strip()

In [9]:
pred_df = old_pred_df.merge(ownership_df_name, how='outer', on='full_name')

In [21]:
pred_df

,full_name,pos,team_name,total_points,y_pred,notes,y_pred_delta_1,y_pred_delta_2,y_pred_delta_3,y_pred_delta_4,...,y_pred_delta_7,y_pred_delta_8,y_pred_delta_9,y_pred_delta_10,is_available,element,in_accepted_trade,owner,status,id
0,aaron_anselmino,DEF,Chelsea,0.0,-1.800599,NaN,0.625922,0.0,0.000000,0.000000,...,0.000000,8.871476,0.000000,0.000000,True,234.0,False,NaN,a,234.0
1,aaron_hickey,DEF,Brentford,0.0,5.340009,NaN,0.000000,0.0,0.000000,0.611548,...,0.000000,0.000000,0.000000,0.000000,True,116.0,False,NaN,a,116.0
2,aaron_ramsdale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,673.0,False,NaN,a,673.0
3,aaron_ramsey,MID,Burnley,0.0,27.162412,promoted_team,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,True,211.0,False,NaN,a,211.0
4,aaron_wan_bissaka,DEF,West Ham,118.0,56.569787,NaN,0.000000,0.0,0.000000,0.000000,...,1.872975,0.000000,0.000000,0.000000,True,610.0,False,NaN,a,610.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,zain_silcott_duberry,MID,Bournemouth,1.0,24.922663,jan_transfer,0.086670,0.0,0.000000,0.000000,...,0.000000,0.000000,0.421542,0.000000,True,95.0,False,NaN,a,95.0
704,zak_johnson,DEF,Sunderland,0.0,33.661237,promoted_team,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.625922,True,538.0,False,NaN,a,538.0
705,zeki_amdouni,FWD,Burnley,0.0,66.568110,promoted_team,0.000000,0.0,0.000000,0.970461,...,5.550651,1.220135,0.000000,0.356870,True,216.0,False,NaN,a,216.0
706,zepiqueno_redmond,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,65.0,False,NaN,a,65.0


In [12]:
pos_pred_df2 = pred_df.sort_values('y_pred', ascending=False).query('status=="a"')

pos_pred_df3 = pos_pred_df2.iloc[:, :5]

In [13]:
updated_player_df = get_latest_players_df()

col_of_interest = ['first_name', 'second_name',
                   'form', 'total_points', 'transfers_in', 'transfers_out', 'value_form', 'minutes']

current_stats = updated_player_df[col_of_interest]

current_stats['full_name'] = updated_player_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)

current_stats2 = current_stats.drop(['first_name', 'second_name'], axis=1)

current_stats2['transfer_delta'] = current_stats2['transfers_in'] - current_stats2['transfers_out']

C:\Users\dellis2\AppData\Local\Temp\ipykernel_32608\1447550088.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_stats['full_name'] = updated_player_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)


In [14]:
pos_pred_df4 = pos_pred_df3.merge(current_stats2, on='full_name', how='left')

In [24]:
pos_pred_df4.query('pos=="MID"').head(30)

,full_name,pos,team_name,total_points_x,y_pred,form,total_points_y,transfers_in,transfers_out,value_form,minutes,transfer_delta
0,son_heung_min,MID,Spurs,129.0,165.209198,0.0,0,0,20555,0.0,0,-20555
1,luis_diaz_marulanda,MID,Liverpool,183.0,144.874270,0.0,0,0,5658,0.0,0,-5658
2,marcus_rashford,MID,Man Utd,95.0,117.646442,0.0,0,0,1275,0.0,0,-1275
3,justin_kluivert,MID,Bournemouth,158.0,115.192051,0.5,1,7668,42991,0.1,12,-35323
5,savio_moreira_de_oliveira,MID,Man City,106.0,111.910340,0.0,0,437,61063,0.0,0,-60626
6,james_maddison,MID,Spurs,133.0,111.047565,0.0,0,278,15933,0.0,0,-15655
8,iliman_ndiaye,MID,Everton,114.0,101.479645,5.0,10,84900,261738,0.8,153,-176838
10,amad_diallo,MID,Man Utd,129.0,99.592564,1.0,2,19911,93082,0.2,86,-73171
11,alejandro_garnacho_ferreyra,MID,Man Utd,106.0,99.592564,0.0,0,1758,7254,0.0,0,-5496
13,ismaila_sarr,MID,Crystal Palace,149.0,98.227482,5.5,11,138727,104998,0.8,180,33729


In [17]:
pos_pred_df4.sort_values('transfers_in', ascending=False).query('pos=="DEF"').head(50)

,full_name,pos,team_name,total_points_x,y_pred,form,total_points_y,transfers_in,transfers_out,value_form,minutes,transfer_delta
264,micky_van_de_ven,DEF,Spurs,32.0,44.436177,6.0,12,429320,205504,1.3,180,223816
307,matty_cash,DEF,Aston Villa,76.0,40.406014,5.0,10,359375,92930,1.1,180,266445
117,tino_livramento,DEF,Newcastle,105.0,62.717278,6.5,13,307347,52078,1.3,180,255269
149,rico_lewis,DEF,Man City,66.0,57.743441,6.0,12,300862,76581,1.2,155,224281
391,reinildo_mandava,DEF,Sunderland,0.0,33.661237,4.5,9,256900,88762,1.1,180,168138
540,josh_acheampong,DEF,Chelsea,4.0,-1.800599,4.5,9,241681,74963,1.1,90,166718
221,marc_guehi,DEF,Crystal Palace,116.0,45.849293,4.0,8,221811,123179,0.9,180,98632
379,maxime_esteve,DEF,Burnley,0.0,33.661237,3.5,7,166204,164262,0.9,172,1942
115,dan_burn,DEF,Newcastle,115.0,62.717278,5.5,11,143928,50660,1.1,180,93268
373,gabriel_gudmundsson,DEF,Leeds,0.0,33.661237,4.0,8,131592,57144,1.0,180,74448
